## **프로젝트: 멋진 작사가 만들기**

### **Step 1. 데이터 읽어오기**

In [1]:
# 필요한 라이브러리 import
import glob
import os, re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# 데이터 불러오기
txt_file_path = 'C:/Users/Minjoo Lee/AIFFEL/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus에 담기
for txt_file in txt_list:
    with open(txt_file, 'rt', encoding='UTF8') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough"]


### **Step 2. 데이터 정제**

In [3]:
# 정제 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

# 문장이 어떻게 필터링 되는지 확인
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 정제 데이터 구축
# 정제된 문장을 담을 리스트
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 원하지 않는 문장은 건너뛰기
    if len(sentence) == 0: continue # 길이가 0
    if sentence[-1] == ":": continue # 문장의 끝이 :
    
    # preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과 확인
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

In [5]:
# 데이터를 토큰화하는 함수
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장 완성
    tokenizer.fit_on_texts(corpus)
    # tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 304  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2  23  77 ...   0   0   0]
 [  2  42  26 ...   0   0   0]
 [  2  23  77 ...   0   0   0]] <keras.preprocessing.text.Tokenizer object at 0x00000161E67B5648>


생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력해보자.

In [6]:
print(tensor[:3, :10])

[[   2  304   28   99 4811    3    0    0    0    0]
 [   2  221   13   85  226    6  115    3    0    0]
 [   2   24   17 1087 2347    3    0    0    0    0]]


In [7]:
# 단어 사전이 어떻게 구축되었는지 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


2번 인덱스가 `<start>`여서 모든 행이 2로 시작하는 것을 확인할 수 있다.

In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2  304   28   99 4811    3    0    0    0    0    0    0    0    0]
[ 304   28   99 4811    3    0    0    0    0    0    0    0    0    0]


행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩을 채워 넣은 것이다.

### **Step 3. 평가 데이터셋 분리**

In [9]:
# 데이터셋 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=88)

In [10]:
# 데이터셋 크기 확인
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [11]:
# 데이터셋 객체 생성
def create_dataset(src_input,tgt_input):
    BUFFER_SIZE = len(src_input)
    BATCH_SIZE = 256
    steps_per_epoch = len(src_input) // BATCH_SIZE

    VOCAB_SIZE = tokenizer.num_words + 1

    dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

train_dataset = create_dataset(enc_train, dec_train)
valid_dataset = create_dataset(enc_val, dec_val)

### **Step 4. 인공지능 만들기**

In [12]:
# 모델 구현
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size, dropout_rate):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.dropout_1 = tf.keras.layers.Dropout(dropout_rate)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.dropout_2 = tf.keras.layers.Dropout(dropout_rate)
        self.linear = tf.keras.layers.Dense(vocab_size)
    
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.dropout_1(out)
        out = self.rnn_2(out)
        out = self.dropout_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 1024 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기
hidden_size = 2048 # 모델에 얼마나 많은 일꾼을 둘 것인가?
dropout_rate = 0.2
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size, dropout_rate) 

In [13]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in train_dataset.take(1): break
    
# 불러온 데이터를 모델에 넣어보기
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-2.6376138e-04, -4.6119507e-04,  2.8968582e-04, ...,
          2.4202246e-04, -1.0433166e-04, -1.0743681e-04],
        [-2.7145414e-05, -7.8982412e-04,  1.1621646e-03, ...,
          2.1229715e-04, -6.1762439e-05, -1.3538405e-04],
        [ 7.7120661e-05, -8.5416622e-04,  7.7189744e-04, ...,
          5.5689760e-04, -3.5853413e-04,  1.6359851e-04],
        ...,
        [ 1.4803661e-03,  8.1505592e-04, -2.0033547e-04, ...,
         -8.1072416e-04, -1.1499242e-03, -1.6644222e-03],
        [ 1.1851481e-03,  1.0534307e-03,  1.8646722e-04, ...,
         -5.1133573e-04, -6.2646717e-04, -1.3026163e-03],
        [ 8.7161199e-04,  1.0852305e-03,  8.5541855e-05, ...,
         -4.4379581e-04, -1.5149425e-05, -7.0503535e-04]],

       [[-2.6376138e-04, -4.6119507e-04,  2.8968582e-04, ...,
          2.4202246e-04, -1.0433166e-04, -1.0743681e-04],
        [-3.5065092e-04, -6.3639862e-04,  3.0992139e-04, ...,
          1.0990279e-06, 

In [14]:
# 모델 구조 확인
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  12289024  
                                                                 
 lstm (LSTM)                 multiple                  25174016  
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 lstm_1 (LSTM)               multiple                  33562624  
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense (Dense)               multiple                  24590049  
                                                                 
Total params: 95,615,713
Trainable params: 95,615,71

모델은 입력 시퀀스의 길이를 모르기 때문에 Output Shape를 특정할 수 없다.

In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                     reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, 
          validation_data=valid_dataset,
          batch_size=256,
          epochs=10)

Epoch 1/10
549/549 [==============================] - 80s 143ms/step - loss: 3.4340 - val_loss: 3.0322
Epoch 2/10
549/549 [==============================] - 79s 144ms/step - loss: 2.8617 - val_loss: 2.7554
Epoch 3/10
549/549 [==============================] - 80s 146ms/step - loss: 2.5407 - val_loss: 2.5736
Epoch 4/10
549/549 [==============================] - 79s 144ms/step - loss: 2.2545 - val_loss: 2.4413
Epoch 5/10
549/549 [==============================] - 79s 144ms/step - loss: 2.0020 - val_loss: 2.3522
Epoch 6/10
549/549 [==============================] - 79s 144ms/step - loss: 1.7830 - val_loss: 2.2896
Epoch 7/10
549/549 [==============================] - 79s 144ms/step - loss: 1.5999 - val_loss: 2.2494
Epoch 8/10
549/549 [==============================] - 79s 143ms/step - loss: 1.4487 - val_loss: 2.2308
Epoch 9/10
549/549 [==============================] - 80s 145ms/step - loss: 1.3272 - val_loss: 2.2276
Epoch 10/10
549/549 [==============================] - 79s 144ms/step - l

### **Step 5. 평가하기**

In [16]:
# 문장 생성 함수 정의
# 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장 만들기
    while True: # 루프를 돌면서 init_sentence에 단어를 하나씩 생성
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated # 최종적으로 모델이 생성한 문장을 반환

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15)

'<start> i love you more than i love myself , <end> '

---

### **프로젝트 회고**

이번 프로젝트에서의 목표는 10 epoch 안에 val_loss 값을 2.2 이하로 줄이는 것이었다. 그러나 초기 val_loss 값이 이미 2.2에 근접한 값이었기 때문에 embedding_size와 hidden_size를 조절하는 것만으로는 val_loss 값을 쉽게 개선할 수 없었다. 따라서, 추가적인 시도를 해봤다. 첫번째로는 model.fit에 다양한 인자를 추가해봤지만, 이는 val_loss 값에 큰 변화를 가져오지 못했다. 그 다음으로는 dropout 레이어를 추가하고 random_state 숫자를 변경해가며 실험해봤는데, 이렇게 함으로써 val_loss 값을 2.2로 달성할 수 있었다. 그러나 마지막 epoch에서 val_loss 값이 증가하기 시작하는 현상을 관찰할 수 있었고, 이는 과대적합이 발생했을 가능성을 의미한다. 따라서, 과대적합을 완화하기 위해 추가적인 조치가 필요할 것으로 판단된다. 모델 구조 변경, 하이퍼파라미터 튜닝, 정규화 등 여러가지를 시도해봤는데도 불구하고 모델의 성능 향상이 크게 이루어지지 않았는데, LSTM을 대체할 다른 모델을 사용해보는 것도 하나의 방법일 것 같다.

#### **새로 알게 된 것!**
* 시퀀스(Sequence): 연속된 데이터 요소들이 순서대로 나열된 것
* 순환신경망(RNN): 순차적인 데이터를 다루는 인공지능 모델, 이전 단계의 정보를 기억하면서 현재 단계의 입력과 함께 다음 단계의 출력을 계산하는 방식으로 동작
* 언어 모델(Language Model): 이전 단어들의 시퀀스가 주어졌을 때, 다음 단어의 확률을 예측하는 모델

**GPU 설치**  
드.디.어. 로컬 환경에 GPU를 성공적으로 설치했다!!! 이번 프로젝트에서 학습 시간이 너무 오래 걸려서 이전에 실패했던 GPU 설치에 다시 도전해봤다. 그 결과로 확실히 클라우드 서비스 대신 로컬에서 GPU를 사용하면 학습 시간이 단축되고 작업을 보다 빠른 속도로 수행할 수 있었다. 이제 GPU를 통해 딥러닝 작업을 로컬에서 더욱 빠르고 효율적으로 진행할 수 있게 되었다 :)

**참고한 레퍼런스**
<br>[Python] 시퀀스 자료형 #1 리스트(list) https://kukuta.tistory.com/310
<br>[인공지능 개념] Tensor란 무엇인가? https://rekt77.tistory.com/102
<br>tf.keras.Model - TensorFlow https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit
<br>GPU 지원 - TensorFlow https://www.tensorflow.org/install/gpu?hl=ko
<br>[jupyter notebook]tensorflow-gpu 사용 설정하기 https://hengbokhan.tistory.com/75